In [144]:
import geopandas as gpd
import pandas as pd
import json
import seaborn as sns
from tqdm import tqdm_notebook

import random
import matplotlib.pyplot as plt
from functools import partial
import networkx as nx

import copy

## reading in dataframe and adding columns

In [311]:
tx_data = gpd.read_file("TX_vtds/TX_vtds.shp")
tx_data = tx_data[tx_data['TOTPOP'] > 0]

In [146]:
tot_pop = tx_data["TOTPOP"].sum()
print(tot_pop)

ideal_pop = tot_pop/36
print(ideal_pop)

25145561.0
698487.8055555555


In [308]:
def add_columns(state, dem_election, rep_election):
    state["beta"] = (state[dem_election] - state[rep_election])/(state["TOTPOP"])
    state["alpha"] = (state["TOTPOP"] - 2*state[rep_election])
    state["alpha_two"] = (state[dem_election] - 3*state[rep_election] + state["TOTPOP"])
    state["alpha_over_pop"] = (state["TOTPOP"] - 2*state[rep_election])/(state["TOTPOP"])
    state["pr_turnout"] = (state[dem_election] + state[rep_election])/(state["TOTPOP"])

In [313]:
add_columns(tx_data, "PRES16D", "PRES16R")

## greedy function

In [299]:
%%time
def greedy_algorithm_tx(state, plan_column, dem_election, rep_election):
    state = state.sort_values(plan_column, ascending = False)
    state = state.reset_index()
    margin = 0
    gimme_gimme = {}
    for index, row in state.iterrows():
        if margin >= 0:
            margin += (state[dem_election][index] - state[rep_election][index])
            gimme_gimme[index] = (state["TOTPOP"][index], state[dem_election][index], state[rep_election][index])
        else:
            break
    
#tallying up greedy things    
    dems = 0
    reps = 0
    pop = 0 
    for i in range(len(gimme_gimme)):
        dems += gimme_gimme[i][1]
        reps += gimme_gimme[i][2]
        pop += gimme_gimme[i][0]
#calculating seats    
    dem_seats = int(pop/ideal_pop)
    rep_seats = 36 - dem_seats
    print("There are " + str(dem_seats) + " Dem seats and " + str(rep_seats) + " Rep seats.")
#calculating rho
    dem_turnout = 0
    rep_turnout = 0
    for i in range(len(gimme_gimme)):
        dem_turnout += gimme_gimme[i][1] + gimme_gimme[i][2]
    T_dem = dem_turnout/dem_seats
    
    for index, row in state.iterrows():    
        if index > (len(gimme_gimme)):
            rep_turnout += state[dem_election][index] + state[rep_election][index]
    T_rep = rep_turnout/(36 - dem_seats)
    
    print ("Dem turnout is " + str(T_dem) + ", Rep turnout is " + str(T_rep) +  ", rho is " + str(T_rep/T_dem))

    
#defining noise function    
def noise(seats, rho):
    return ((seats/36)*(1 - rho)*(1 - (seats/36)))/((seats/36) * (1 - rho) + rho)

Wall time: 0 ns


In [318]:
%%time
greedy_algorithm_tx(tx_data, "alpha_over_pop", "PRES16D", "PRES16R")

There are 32 Dem seats and 4 Rep seats.
Dem turnout is 224050.71875, Rep turnout is 347389.75, rho is 1.5504960302654687
Wall time: 4.05 s


In [316]:
noise(32, 1.5504960302654687)

-0.05123606178461657

## storing data

In [266]:
col_names = ['method', 'noise term', "hypothetical dem seats", "rho"]
gerrydata = pd.DataFrame(columns = col_names)

In [276]:
gerrydata = gerrydata.append(pd.Series(['beta', -0.056301023499100385, 29, 1.3864482742036979], index=gerrydata.columns ), ignore_index=True)
gerrydata = gerrydata.append(pd.Series(['alpha', -0.08987561240594023, 27, 1.5445980430052453], index = gerrydata.columns), ignore_index = True)
gerrydata = gerrydata.append(pd.Series(['alpha_two', -0.07747260642000688, 29, 1.5472323555660084], index = gerrydata.columns), ignore_index = True)
gerrydata = gerrydata.append(pd.Series(['alpha_over_pop', -0.0820127305414992, 29, 1.5829368248691753], index = gerrydata.columns), ignore_index = True)
gerrydata = gerrydata.append(pd.Series(['actual 2012 pres elections', -0.0769170266661768, 11, 1.4844350553571108], index = gerrydata.columns), ignore_index = True)

In [277]:
gerrydata

,method,noise term,hypothetical dem seats,rho
0,beta,-0.056301,29,1.386448
1,alpha,-0.089876,27,1.544598
2,alpha_two,-0.077473,29,1.547232
3,alpha_over_pop,-0.082013,29,1.582937
4,actual 2012 pres elections,-0.076917,11,1.484435


## extra functions (seat count, district cutter-upper)

In [180]:
tx_data["USCD"] = tx_data["USCD"].astype(int)

In [98]:
#finding actual noise and seat share of a plan
def seat_count_turnout_real(state, dem, rep):
    dem_votes = [0]*36
    rep_votes = [0]*36
    tot_votes = [0]*36
    tot_dem_won = 0
    tot_rep_won = 0
    dem_seats = 0
    rep_seats = 0
    for i in range(36):
        for index, row in state.iterrows():
             if state["USCD"][index] == i+1:
                dem_votes[i] += state[dem][index]
                rep_votes[i] += state[rep][index]
                tot_votes[i] += state[dem][index] + state[rep][index]
    for j in range(36):
        if dem_votes[j] > rep_votes[j]:
            dem_seats += 1
            tot_dem_won += tot_votes[j]
        else:
            rep_seats += 1
            tot_rep_won += tot_votes[j]
    
    return (dem_seats, rep_seats, 
            "T_dem is " + str(float(tot_dem_won/dem_seats)), 
            "T_rep is " + str(float(tot_rep_won/rep_seats)), 
            "rho is " + str(float((tot_rep_won/rep_seats)/(tot_dem_won/dem_seats))))

In [278]:
%%time
seat_count_turnout_real(tx_data, "PRES12D", "PRES12R")

Wall time: 1min 46s


(11,
 25,
 'T_dem is 163697.45454545456',
 'T_rep is 242998.24',
 'rho is 1.4844350553571108')

In [17]:
def seat_count_turnout(statedf):
    turnouts = [0]*36
    margins = [0]*36
    dem_seats = 0
    rep_seats = 0
    av_dem = 0
    av_rep = 0
    for i in range(36):
        for index, row in statedf.iterrows():
             if statedf["gerry_districts"][index] == i:
                turnouts[i] += statedf["pr_turnout"][index]
                margins[i] += (statedf["PRES12D"][index] - statedf["PRES12R"][index])   
    for j in range(36):
        if margins[j] >0:
            dem_seats += 1
            av_dem += turnouts[j]
        else:
            rep_seats += 1
            av_rep += turnouts[j]
    return (dem_seats, rep_seats, av_dem/dem_seats, av_rep/rep_seats)

In [78]:
#sorting by a metric and then splitting into districts
def cheat_eg(state, plan_column, num_dist, pop_label):
    state = state.sort_values(plan_column, ascending = False)
    state = state.reset_index()
    tot_pop = state[pop_label].sum()
    print(tot_pop)
    ideal_pop = tot_pop/num_dist - 300
    print(ideal_pop + 300)
    
    districts = {}
    d_counter = 0
    p_counter = 0
    for index, row in state.iterrows():
        if p_counter <= ideal_pop:
            p_counter += row[pop_label]
            districts[index] = d_counter
        else:
            p_counter = 0
            d_counter += 1
            districts[index] = d_counter
            
    state['gerry_districts'] = state.index.map(districts)
    return state
    